__качаем объявления о продаже квартир с циан.ru__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
import sys
import re
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd
from bs4 import BeautifulSoup
from time import sleep
from random import random

In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

In [3]:
raw_data_path = 'data/raw/flat/cian' # каталог с файлами объявлений

In [4]:
profile_path = '/home/mechanoid/.mozilla/firefox/p144xo2m.default-release'
# base_url, req_param = 'https://sevastopol.cian.ru/kupit-kvartiru/', ''
place = 'sevastopol'
base_url, req_param ='https://www.cian.ru/cat.php', 'deal_type=sale&offer_type=flat&region=184723'

In [5]:
%%time

# from lib.downloader import DownloaderSimple
# from lib.parser import AdsListParser
from lib.downloader import DownloaderSeleniumFirefox
from lib.cian import CianParser


df,html = CianParser(
        base_url = base_url,
        driver=DownloaderSeleniumFirefox(profile_path),
        # driver=DownloaderSimple(),
    ).load(req_param=req_param, keep_html=True) # , npage_start=53) # , page_limit=3 ) #

print(len(df))

[INFO    ] 2022-11-28 15:21:37 | DownloaderSeleniumFirefox: downloader init
[INFO    ] 2022-11-28 15:21:37 | DownloaderSeleniumFirefox: open virtual browser
[INFO    ] 2022-11-28 15:21:39 | AdsListParser: downloader init
[INFO    ] 2022-11-28 15:21:39 | CianParser: downloader init
[INFO    ] 2022-11-28 15:21:39 | AdsListParser: start read and parse pages...
[INFO    ] 2022-11-28 15:21:53 | AdsListParser: read page 1
[INFO    ] 2022-11-28 15:22:09 | AdsListParser: read page 2
[INFO    ] 2022-11-28 15:22:18 | AdsListParser: read page 3
[INFO    ] 2022-11-28 15:22:27 | AdsListParser: read page 4
[INFO    ] 2022-11-28 15:22:32 | AdsListParser: read page 5
[INFO    ] 2022-11-28 15:22:38 | AdsListParser: read page 6
[INFO    ] 2022-11-28 15:22:46 | AdsListParser: read page 7
[INFO    ] 2022-11-28 15:22:54 | AdsListParser: read page 8
[INFO    ] 2022-11-28 15:23:06 | AdsListParser: read page 9
[INFO    ] 2022-11-28 15:23:16 | AdsListParser: read page 10
[INFO    ] 2022-11-28 15:23:25 | AdsLis

In [6]:
df.sample(3)

,OfferTitle,OfferSubtitle,Deadline,MainPrice,PriceInfo,GeoLabel,TimeLabel,LinkArea,Description,page,ts
1094,"2-комн. кв., 62 м², 8/10 этаж",,,8 200 000 ₽,132 258 ₽/м²,"[Севастополь, р-н Гагаринский, улица Военных Строителей, 1В/1]","[вчера, вчера, 12:50]",https://sevastopol.cian.ru/sale/flat/271724673/,"Продается двухкомнатная квартира 62 м2, расположенная на восьмом этаже десятиэтажного дома. \nТехнические характеристики: \n \n-Год постройки дома: 2022\n-Этаж /этажность: 8/10\n-Количество комнат: 2\n-Комнаты: изолированные\n-Общая площадь: 62 м2\n-Жилая площадь: 32,28 м2\n-Площадь кухни: 9 м2\n-Высота потолка: 2,8 м\n-Санузел: совмещенный\n-Водоснабжение: центральный\n-Горячее водоснабжение: АГВ\n-Отопление: АГВ\nДополнительная информация: \nКорпус 7.4 ЖСК Скифия ближайший к морю. Из окон открываются потрясающие виды на бухты, город и открытое море.\n Площадь комнат 17 и 15,5 м2.\nПлощадь кухни - 9 м2.\nЕсть просторная прихожая площадью 8 м2.\nВысота потолков в квартире 2,8 м.\nИз большой комнаты есть выход на просторный балкон-веранду площадью 6 м2, а также на лоджию-веранду 3 м2. Окна квартиры выходят на разные стороны дома.\nПанорамные окна балкона открывают потрясающие виды на Казачью и Соленую бухты, мыс Херсонес с маяком и открытое море. \nМеталлопластиковые окна, металлическая входная дверь, автономное отопление и горячее водоснабжение (двухконтурный газовый котел) с разводкой труб, а также установка приборов учета, дополняют список преимуществ этого прогрессивного жилого комплекса.\nКонструктив дома выполнен из монолитного железобетонного каркаса, стены - из газоблока, фасад утеплен пенополистиролом, крыша - шатровая двускатная.\nСовременный жилой комплекс ""Скифия"" в быстро развивающемся микрорайоне со сейсмостойкостью зданий 8 баллов набирает все бОльшую популярность у жителей и гостей города. \nВсе, что необходимо человеку для самой комфортной и перспективной жизни в Севастополе, здесь есть (школа, детские сады, рынок, отличные пляжи и др.).\nБольшая придомовая территория благоустроена спортивными и игровыми площадками для детей, наземной парковкой, зонами для отдыха для детей и взрослых.\nВ дополнение к квартире продаются 2 подземных гаража площадью 17,5 м2 каждый, которые также расположены в ЖК Скифия. Гаражи оснащены электричеством (установлен счетчик на потребление), пожарной сигнализацией, рольставнями.\nСдача дома в эксплуатацию планируется в 3 квартале 2022 года, квартира продается по переуступке пая, что выгодно для покупателя.\nПозвоните нам, чтобы назначить просмотр! \nКвартира продается по переуступке. Рассматривается только наличный расчёт. \nДокументы РФ, проверены юристами и готовы к сделке. \nПрофессиональное сопровождение до получения права собственности.\nГотовы к быстрому выходу на сделку.",40,2022-11-28 15:30:48.195785
716,"1-комн. кв., 32,3 м², 5/16 этаж",,,5 390 000 ₽,166 873 ₽/м²,"[Севастополь, р-н Гагаринский, мкр. 5-й, улица Александра Маринеско, 1Ак1]","[вчера, вчера, 12:50]",https://sevastopol.cian.ru/sale/flat/276585179/,"Однокомнатная квартира 32,3 м2, на 5/16 этаже дома. Расположена в г. Севастополь, Гагаринский район, ул. Александра Маринеско, д. 1 А.\nПозвоните нам, чтобы узнать все детали! \nТехнические характеристики: \n \n-Адрес: г. Севастополь, Гагаринский район, ул. Александра Маринеско, д. 1 А\n-Год постройки дома: 2021\n-Этаж/этажность: 5/16\n-Количество комнат: 1\n-Комнаты: изолированные \n-Общая площадь: 32,3 м2\n-Жилая площадь: 16,71 м2\n-Комната: 16,71 м2\n-Площадь кухни: 5 м2\n-Высота потолка: 2,7 м \n-Санузел: совмещенный \n-Водоснабжение: центральное\n-Горячее водоснабжение: бойлер\n-Отопление: АГВ\nДополнительная информация: \nОтличная квартира-студия в ЖК Гагаринские Высотки.\nОднокомнатная просторная квартира-студия площадью 32.3 м2 находятся на 5 этаже первого 16-этажного корпуса. Удачная планировка позволяет комфортно разместиться. В квартире предусмотрено место для гардеробной, также панорамное окно с видом на город и море. \nПозвоните нам, чтобы назначить просмот

In [7]:
assert len(df)>0

In [8]:
print(len(df))
df = pd.DataFrame(df).dropna().drop_duplicates(['LinkArea','Description',]).reset_index(drop=True)
print(len(df))

1500
1424


In [9]:
df['place']=place

In [10]:
ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
assert len(df)>0
# df.to_excel(f'{raw_data_path}/cian_{ts}_raw.xlsx',index=False)
df.to_pickle(f'{raw_data_path}/cian_{ts}_raw.pkl')

----

In [11]:
# df = pd.read_excel('data/raw/cian/cian_2022-09-08_13-49_raw.xlsx')

In [12]:
# df

In [13]:
# df['GeoLabel'].apply( lambda s : len(s) ) #', '.join(s) )

# s = ['Севастополь', 'р-н Гагаринский', 'Казачья Бухта мкр', 'Скифия ЖК']
# ', '.join(s)

In [14]:
# from lib.cian import CianDataCleaner

# CianDataCleaner().transform(df)

----

In [15]:
# with open('tmp/cian.html','wt') as f: f.write(html[0])